# 성능튜닝

## 1.환경준비

### (1) import

In [1]:
#라이브러리들을 불러오자.
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

# 전처리
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler

# 모델링
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier, plot_tree
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.metrics import * 

import warnings    # 경고메시지 제외
warnings.filterwarnings(action='ignore')

### (2) 데이터 준비

* 변수설명
    * COLLEGE : 대학 졸업여부
    * INCOME : 연수입
    * OVERAGE : 월평균 초과사용 시간(분)
    * LEFTOVER : 월평균 잔여시간비율(%)
    * HOUSE : 집값
    * HANDSET_PRICE : 스마트폰 가격
    * OVER_15MINS_CALLS_PER_MONTH : 월평균 장기통화(15분이상) 횟수
    * AVERAGE_CALL_DURATION : 평균 통화 시간
    * REPORTED_SATISFACTION : 만족도 설문조사 결과
    * REPORTED_USAGE_LEVEL : 사용도 자가진단 결과
    * CONSIDERING_CHANGE_OF_PLAN : 향후 변경계획 설문조사 결과
    * CHURN : 이탈(번호이동) 여부 (1-이탈, 0-잔류, Target 변수)


In [2]:
# 데이터를 불러옵시다.
path = 'https://raw.githubusercontent.com/DA4BAM/dataset/master/mobile_cust_churn.csv'
data = pd.read_csv(path)
data = data.sample(5000, random_state = 2022)
data['CHURN'] = data['CHURN'].map({'LEAVE':1, 'STAY':0})
data.head()

,id,COLLEGE,INCOME,OVERAGE,LEFTOVER,HOUSE,HANDSET_PRICE,OVER_15MINS_CALLS_PER_MONTH,AVERAGE_CALL_DURATION,REPORTED_SATISFACTION,REPORTED_USAGE_LEVEL,CONSIDERING_CHANGE_OF_PLAN,CHURN
3178,3179,0,119512,51,31,248566,229,5,2,very_sat,very_high,considering,1
14926,14927,1,142144,192,15,774317,581,29,4,unsat,very_little,never_thought,1
15116,15117,1,142308,0,79,306426,497,1,1,sat,little,considering,0
12733,12734,1,113385,0,0,333599,819,1,6,very_unsat,very_high,considering,1
14032,14033,1,90348,209,10,637286,360,26,4,unsat,little,actively_looking_into_it,0


## 2.데이터 준비

### (1) 데이터 정리

In [3]:
drop_cols = ['id']
data.drop(drop_cols, axis = 1, inplace = True )

### (2) 데이터분할1 : x, y 나누기

In [4]:
target = 'CHURN'
x = data.drop(target, axis = 1)
y = data.loc[:, target]

### (3) NA 조치

### (4) 가변수화

In [5]:
dumm_cols = ['REPORTED_SATISFACTION','REPORTED_USAGE_LEVEL','CONSIDERING_CHANGE_OF_PLAN']
x = pd.get_dummies(x, columns = dumm_cols, drop_first = True)

### (5) 데이터분할2 : train : validation 나누기

In [6]:
x_train, x_val, y_train, y_val = train_test_split(x, y, test_size = .3, random_state = 20)

### (6) Scaling

In [7]:
scaler = MinMaxScaler()
x_train_s = scaler.fit_transform(x_train)
x_val_s = scaler.transform(x_val)

## 3.선형모델 튜닝

Logistic Regression : 전진선택법
* 변수를 하나씩 늘려가면서
* AIC를 가장 낮추는 모델 찾기

### (1) 전진선택을 수행할 함수 만들기( **로지스틱 회귀** 용)

In [8]:
# 아래 함수는 로지스틱 회귀를 위한 전진선택법 함수 입니다.
import statsmodels.api as sm

def forward_stepwise_logistic(x_train, y_train):

    # 변수목록, 선택된 변수 목록, 단계별 모델과 AIC 저장소 정의
    features = list(x_train)
    selected = []
    step_df = pd.DataFrame({ 'step':[], 'feature':[],'aic':[]})

    # 
    for s in range(0, len(features)) :
        result =  { 'step':[], 'feature':[],'aic':[]}

        # 변수 목록에서 변수 한개씩 뽑아서 모델에 추가
        for f in features :
            vars = selected + [f]
            x_tr = x_train[vars]
            model = sm.Logit(y_train, x_tr).fit()
            result['step'].append(s+1)
            result['feature'].append(vars)
            result['aic'].append(model.aic)
        
        # 모델별 aic 집계
        temp = pd.DataFrame(result).sort_values('aic').reset_index(drop = True)

        # 만약 이전 aic보다 새로운 aic 가 크다면 멈추기
        if step_df['aic'].min() < temp['aic'].min() :
            break
        step_df = pd.concat([step_df, temp], axis = 0).reset_index(drop = True)

        # 선택된 변수 제거
        v = temp.loc[0,'feature'][s]
        features.remove(v)

        selected.append(v)
    
    # 선택된 변수와 step_df 결과 반환
    return selected, step_df

### (2) 전진선택법 수행

In [9]:
vars, result = forward_stepwise_logistic(x_train, y_train)

Optimization terminated successfully.
         Current function value: 0.693075
         Iterations 3
Optimization terminated successfully.
         Current function value: 0.693062
         Iterations 2
Optimization terminated successfully.
         Current function value: 0.683528
         Iterations 4
Optimization terminated successfully.
         Current function value: 0.693007
         Iterations 3
Optimization terminated successfully.
         Current function value: 0.684909
         Iterations 2
Optimization terminated successfully.
         Current function value: 0.693042
         Iterations 2
Optimization terminated successfully.
         Current function value: 0.686899
         Iterations 4
Optimization terminated successfully.
         Current function value: 0.692565
         Iterations 3
Optimization terminated successfully.
         Current function value: 0.692806
         Iterations 4
Optimization terminated successfully.
         Current function value: 0.693137
  

Optimization terminated successfully.
         Current function value: 0.633162
         Iterations 5
Optimization terminated successfully.
         Current function value: 0.633092
         Iterations 5
Optimization terminated successfully.
         Current function value: 0.633175
         Iterations 5
Optimization terminated successfully.
         Current function value: 0.633193
         Iterations 5
Optimization terminated successfully.
         Current function value: 0.633016
         Iterations 5
Optimization terminated successfully.
         Current function value: 0.633178
         Iterations 5
Optimization terminated successfully.
         Current function value: 0.633151
         Iterations 5
Optimization terminated successfully.
         Current function value: 0.633158
         Iterations 5
Optimization terminated successfully.
         Current function value: 0.632593
         Iterations 5
Optimization terminated successfully.
         Current function value: 0.632566
  

* 선택된 변수

In [10]:
vars

['OVERAGE',
 'HOUSE',
 'HANDSET_PRICE',
 'LEFTOVER',
 'REPORTED_SATISFACTION_very_sat',
 'INCOME',
 'REPORTED_SATISFACTION_sat']

In [11]:
list(x_train)

['COLLEGE',
 'INCOME',
 'OVERAGE',
 'LEFTOVER',
 'HOUSE',
 'HANDSET_PRICE',
 'OVER_15MINS_CALLS_PER_MONTH',
 'AVERAGE_CALL_DURATION',
 'REPORTED_SATISFACTION_sat',
 'REPORTED_SATISFACTION_unsat',
 'REPORTED_SATISFACTION_very_sat',
 'REPORTED_SATISFACTION_very_unsat',
 'REPORTED_USAGE_LEVEL_high',
 'REPORTED_USAGE_LEVEL_little',
 'REPORTED_USAGE_LEVEL_very_high',
 'REPORTED_USAGE_LEVEL_very_little',
 'CONSIDERING_CHANGE_OF_PLAN_considering',
 'CONSIDERING_CHANGE_OF_PLAN_never_thought',
 'CONSIDERING_CHANGE_OF_PLAN_no',
 'CONSIDERING_CHANGE_OF_PLAN_perhaps']

In [12]:
result

,step,feature,aic
0,1.0,[OVERAGE],4786.699456
1,1.0,[HOUSE],4796.363859
2,1.0,[OVER_15MINS_CALLS_PER_MONTH],4810.294604
3,1.0,[REPORTED_SATISFACTION_very_sat],4845.064834
4,1.0,[AVERAGE_CALL_DURATION],4849.951663
...,...,...,...
114,7.0,"[OVERAGE, HOUSE, HANDSET_PRICE, LEFTOVER, REPO...",4442.309949
115,7.0,"[OVERAGE, HOUSE, HANDSET_PRICE, LEFTOVER, REPO...",4442.315315
116,7.0,"[OVERAGE, HOUSE, HANDSET_PRICE, LEFTOVER, REPO...",4442.316816
117,7.0,"[OVERAGE, HOUSE, HANDSET_PRICE, LEFTOVER, REPO...",4442.448187


In [13]:
pd.set_option('display.width', None)
pd.set_option('display.max_colwidth', -1)

### (3) 모델링

* 전체 변수 

In [14]:
m1 = LogisticRegression()
m1.fit(x_train, y_train)
p1 = m1.predict(x_val)

print(accuracy_score(y_val, p1))
print(classification_report(y_val, p1))

0.6333333333333333
              precision    recall  f1-score   support

           0       0.62      0.68      0.65       738
           1       0.65      0.59      0.62       762

    accuracy                           0.63      1500
   macro avg       0.63      0.63      0.63      1500
weighted avg       0.64      0.63      0.63      1500



* 전진선택법 변수

In [15]:
m2 = LogisticRegression()
m2.fit(x_train[vars], y_train)
p2 = m2.predict(x_val[vars])

print(accuracy_score(y_val, p2))
print(classification_report(y_val, p2))

0.634
              precision    recall  f1-score   support

           0       0.62      0.68      0.65       738
           1       0.66      0.59      0.62       762

    accuracy                           0.63      1500
   macro avg       0.64      0.63      0.63      1500
weighted avg       0.64      0.63      0.63      1500



## 4.하이퍼파라미터 튜닝

### (1) 필요한 함수 불러오기 

In [16]:
from sklearn.model_selection import RandomizedSearchCV, GridSearchCV

### (2) Random Search

① 값의 범위를 지정한다.  
② 모델 선언(시도 횟수 지정)  
③ 모델링(값의 범위 내에서 시도 횟수만큼 랜덤하게 선택해서 시도한다.)  
④ 가장 성능이 좋은 값을 선정


#### ① 값의 범위를 지정한다.

In [17]:
# dictionary형태로 선언
params = { 'n_neighbors' : range(1,51), 'metric' : ['euclidean', 'manhattan']  }
params

{'n_neighbors': range(1, 51), 'metric': ['euclidean', 'manhattan']}

#### ② 모델 선언

In [18]:
# 기본모델
model = KNeighborsClassifier()

# Random Search 설정.
model_rs = RandomizedSearchCV(model
                            , params              # hyperparameter 범위 지정.
                            , cv=5                    # k-fold Cross Validation
                            , n_iter=5                # Random하게 시도할 횟수
                            )

#### ③ 모델링

In [19]:
# 학습 : model이 아니라 model_rs
model_rs.fit(x_train_s, y_train)

RandomizedSearchCV(cv=5, estimator=KNeighborsClassifier(), n_iter=5,
                   param_distributions={'metric': ['euclidean', 'manhattan'],
                                        'n_neighbors': range(1, 51)})

In [20]:
# 튜닝 결과
model_rs.cv_results_

{'mean_fit_time': array([0.00447483, 0.00280137, 0.00144353, 0.00198498, 0.00178161]),
 'std_fit_time': array([0.00193728, 0.00075175, 0.00092554, 0.00013105, 0.00039308]),
 'mean_score_time': array([0.1660162 , 0.15100846, 0.13047204, 0.10301805, 0.10071874]),
 'std_score_time': array([0.01891863, 0.00980646, 0.02764362, 0.00935447, 0.00337004]),
 'param_n_neighbors': masked_array(data=[19, 41, 25, 34, 22],
              mask=[False, False, False, False, False],
        fill_value='?',
             dtype=object),
 'param_metric': masked_array(data=['euclidean', 'manhattan', 'manhattan', 'euclidean',
                    'euclidean'],
              mask=[False, False, False, False, False],
        fill_value='?',
             dtype=object),
 'params': [{'n_neighbors': 19, 'metric': 'euclidean'},
  {'n_neighbors': 41, 'metric': 'manhattan'},
  {'n_neighbors': 25, 'metric': 'manhattan'},
  {'n_neighbors': 34, 'metric': 'euclidean'},
  {'n_neighbors': 22, 'metric': 'euclidean'}],
 'split0_

In [21]:
model_rs.cv_results_['params']

[{'n_neighbors': 19, 'metric': 'euclidean'},
 {'n_neighbors': 41, 'metric': 'manhattan'},
 {'n_neighbors': 25, 'metric': 'manhattan'},
 {'n_neighbors': 34, 'metric': 'euclidean'},
 {'n_neighbors': 22, 'metric': 'euclidean'}]

In [22]:
model_rs.cv_results_['mean_test_score']

array([0.57914286, 0.62228571, 0.61142857, 0.59257143, 0.58685714])

In [23]:
# 최적의 파라미터
model_rs.best_params_

{'n_neighbors': 41, 'metric': 'manhattan'}

In [24]:
# 그때의 성능
model_rs.best_score_

0.6222857142857142

In [25]:
# best 모델로 예측 및 평가
pred = model_rs.predict(x_val_s)
# 가장 성능 좋은 것으로 시도해줌
print(classification_report(y_val, pred))

              precision    recall  f1-score   support

           0       0.59      0.74      0.66       738
           1       0.67      0.51      0.58       762

    accuracy                           0.62      1500
   macro avg       0.63      0.63      0.62      1500
weighted avg       0.63      0.62      0.62      1500



### (3) 실습 : Random Search

* decision tree로 튜닝을 시도해 봅시다.
    * max_depth : 1~10
    * min_samples_leaf : 10 ~ 100

#### ① 값의 범위를 지정한다.

In [26]:
params = {'max_depth': range(1,11),'min_samples_leaf': range(10,101,10)}
params

{'max_depth': range(1, 11), 'min_samples_leaf': range(10, 101, 10)}

#### ② 모델 선언

In [27]:
from sklearn.tree import DecisionTreeClassifier, plot_tree
from sklearn.metrics import *

In [28]:
modelD = DecisionTreeClassifier()

In [29]:
model_rs = RandomizedSearchCV(modelD,params,cv=5,n_iter=5)

In [30]:
model_rs

RandomizedSearchCV(cv=5, estimator=DecisionTreeClassifier(), n_iter=5,
                   param_distributions={'max_depth': range(1, 11),
                                        'min_samples_leaf': range(10, 101, 10)})

#### ③ 모델링

In [31]:
model_rs.fit(x_train,y_train)

RandomizedSearchCV(cv=5, estimator=DecisionTreeClassifier(), n_iter=5,
                   param_distributions={'max_depth': range(1, 11),
                                        'min_samples_leaf': range(10, 101, 10)})

In [32]:
model_rs.cv_results_

{'mean_fit_time': array([0.00980906, 0.00758047, 0.01250014, 0.01868687, 0.01972041]),
 'std_fit_time': array([0.00270434, 0.00175586, 0.00194721, 0.00081372, 0.00307164]),
 'mean_score_time': array([0.00446248, 0.00367055, 0.00512891, 0.00475054, 0.00355086]),
 'std_score_time': array([0.00142455, 0.00093561, 0.00061427, 0.00081872, 0.0006791 ]),
 'param_min_samples_leaf': masked_array(data=[90, 70, 50, 60, 40],
              mask=[False, False, False, False, False],
        fill_value='?',
             dtype=object),
 'param_max_depth': masked_array(data=[1, 1, 2, 5, 8],
              mask=[False, False, False, False, False],
        fill_value='?',
             dtype=object),
 'params': [{'min_samples_leaf': 90, 'max_depth': 1},
  {'min_samples_leaf': 70, 'max_depth': 1},
  {'min_samples_leaf': 50, 'max_depth': 2},
  {'min_samples_leaf': 60, 'max_depth': 5},
  {'min_samples_leaf': 40, 'max_depth': 8}],
 'split0_test_score': array([0.64285714, 0.64285714, 0.66571429, 0.72142857, 0.71

In [33]:
model_rs.best_params_

{'min_samples_leaf': 40, 'max_depth': 8}

In [34]:
model_rs.best_score_

0.694

In [35]:
# best 모델로 예측 및 평가
pred = model_rs.predict(x_val)
# 가장 성능 좋은 것으로 시도해줌
print(classification_report(y_val, pred))

              precision    recall  f1-score   support

           0       0.68      0.69      0.69       738
           1       0.70      0.69      0.69       762

    accuracy                           0.69      1500
   macro avg       0.69      0.69      0.69      1500
weighted avg       0.69      0.69      0.69      1500



### (4) Grid Search

① 값의 범위를 지정한다.  
② 모델링(값의 범위 내에서 모든 조합을 다 시도한다.)  
③ 가장 성능이 좋은 값을 선정


#### ① 값의 범위를 지정한다.

In [36]:
# dictionary형태로 선언
params = { 'n_neighbors' : range(3,31,2), 'metric' : ['euclidean', 'manhattan']  }
params

{'n_neighbors': range(3, 31, 2), 'metric': ['euclidean', 'manhattan']}

#### ② 모델 선언

In [37]:
# 기본모델
model = KNeighborsClassifier()

# Random Search 설정.
model_gs = GridSearchCV(model, params, cv=5)

#### ③ 모델링

In [38]:
# 학습 : model이 아니라 model_rs
model_gs.fit(x_train_s, y_train)

GridSearchCV(cv=5, estimator=KNeighborsClassifier(),
             param_grid={'metric': ['euclidean', 'manhattan'],
                         'n_neighbors': range(3, 31, 2)})

In [39]:
# 튜닝 결과
model_gs.cv_results_

{'mean_fit_time': array([0.00216293, 0.00194998, 0.00244989, 0.00198245, 0.00225048,
        0.00176353, 0.00202093, 0.00156794, 0.00188403, 0.00239239,
        0.00194135, 0.00221295, 0.00160685, 0.00386181, 0.00149136,
        0.00124965, 0.00130343, 0.00141072, 0.00160418, 0.00119534,
        0.00180521, 0.00179558, 0.00190187, 0.00166116, 0.00179334,
        0.00120363, 0.00138535, 0.00117393]),
 'std_fit_time': array([4.18500146e-04, 7.11006807e-04, 5.63495512e-04, 4.23295438e-04,
        3.86410939e-04, 3.88828510e-04, 3.31182545e-05, 5.25015741e-04,
        2.10592333e-04, 1.01872529e-03, 4.32903373e-04, 3.98401124e-04,
        7.79394129e-04, 4.75201461e-03, 4.14724918e-04, 4.04826030e-04,
        3.94849874e-04, 4.61836493e-04, 4.95419869e-04, 3.63861704e-04,
        7.13845366e-04, 3.99256137e-04, 1.80621862e-04, 7.64837550e-04,
        3.96630483e-04, 3.95228100e-04, 4.71250388e-04, 4.28483875e-04]),
 'mean_score_time': array([0.0938858 , 0.10036721, 0.10077372, 0.09826365, 

In [40]:
model_gs.cv_results_['params']

[{'metric': 'euclidean', 'n_neighbors': 3},
 {'metric': 'euclidean', 'n_neighbors': 5},
 {'metric': 'euclidean', 'n_neighbors': 7},
 {'metric': 'euclidean', 'n_neighbors': 9},
 {'metric': 'euclidean', 'n_neighbors': 11},
 {'metric': 'euclidean', 'n_neighbors': 13},
 {'metric': 'euclidean', 'n_neighbors': 15},
 {'metric': 'euclidean', 'n_neighbors': 17},
 {'metric': 'euclidean', 'n_neighbors': 19},
 {'metric': 'euclidean', 'n_neighbors': 21},
 {'metric': 'euclidean', 'n_neighbors': 23},
 {'metric': 'euclidean', 'n_neighbors': 25},
 {'metric': 'euclidean', 'n_neighbors': 27},
 {'metric': 'euclidean', 'n_neighbors': 29},
 {'metric': 'manhattan', 'n_neighbors': 3},
 {'metric': 'manhattan', 'n_neighbors': 5},
 {'metric': 'manhattan', 'n_neighbors': 7},
 {'metric': 'manhattan', 'n_neighbors': 9},
 {'metric': 'manhattan', 'n_neighbors': 11},
 {'metric': 'manhattan', 'n_neighbors': 13},
 {'metric': 'manhattan', 'n_neighbors': 15},
 {'metric': 'manhattan', 'n_neighbors': 17},
 {'metric': 'manha

In [41]:
model_gs.cv_results_['mean_test_score']

array([0.57885714, 0.57942857, 0.57714286, 0.57342857, 0.568     ,
       0.57457143, 0.58085714, 0.58257143, 0.57914286, 0.58057143,
       0.586     , 0.58457143, 0.57971429, 0.58028571, 0.57314286,
       0.59142857, 0.59171429, 0.59142857, 0.59828571, 0.59771429,
       0.59942857, 0.60142857, 0.60171429, 0.61057143, 0.61      ,
       0.61142857, 0.61857143, 0.618     ])

In [42]:
# 최적의 파라미터
model_gs.best_params_

{'metric': 'manhattan', 'n_neighbors': 27}

In [43]:
# 그때의 성능
model_gs.best_score_

0.6185714285714285

In [44]:
# best 모델로 예측 및 평가
pred = model_gs.predict(x_val_s)
print(classification_report(y_val, pred))

              precision    recall  f1-score   support

           0       0.59      0.72      0.65       738
           1       0.65      0.51      0.57       762

    accuracy                           0.61      1500
   macro avg       0.62      0.61      0.61      1500
weighted avg       0.62      0.61      0.61      1500



### (5) 실습 : Grid Search

* decision tree로 튜닝을 시도해 봅시다.
    * max_depth : 1~10
    * min_samples_leaf : 10 ~ 100

#### ① 값의 범위를 지정한다.

In [46]:
params = {'max_depth': range(1,10),'min_samples_leaf' : range(10,101,10)}

#### ② 모델 선언

In [62]:
model = DecisionTreeClassifier()

model_gs = GridSearchCV(model,params,cv=5,verbose=3)

#### ③ 모델링

In [63]:
model_gs.fit(x_train,y_train)

Fitting 5 folds for each of 90 candidates, totalling 450 fits
[CV 1/5] END ...............max_depth=1, min_samples_leaf=10; total time=   0.0s
[CV 2/5] END ...............max_depth=1, min_samples_leaf=10; total time=   0.0s
[CV 3/5] END ...............max_depth=1, min_samples_leaf=10; total time=   0.0s
[CV 4/5] END ...............max_depth=1, min_samples_leaf=10; total time=   0.0s
[CV 5/5] END ...............max_depth=1, min_samples_leaf=10; total time=   0.0s
[CV 1/5] END ...............max_depth=1, min_samples_leaf=20; total time=   0.0s
[CV 2/5] END ...............max_depth=1, min_samples_leaf=20; total time=   0.0s
[CV 3/5] END ...............max_depth=1, min_samples_leaf=20; total time=   0.0s
[CV 4/5] END ...............max_depth=1, min_samples_leaf=20; total time=   0.0s
[CV 5/5] END ...............max_depth=1, min_samples_leaf=20; total time=   0.0s
[CV 1/5] END ...............max_depth=1, min_samples_leaf=30; total time=   0.0s
[CV 2/5] END ...............max_depth=1, min_sa

[CV 5/5] END ...............max_depth=3, min_samples_leaf=30; total time=   0.0s
[CV 1/5] END ...............max_depth=3, min_samples_leaf=40; total time=   0.0s
[CV 2/5] END ...............max_depth=3, min_samples_leaf=40; total time=   0.0s
[CV 3/5] END ...............max_depth=3, min_samples_leaf=40; total time=   0.0s
[CV 4/5] END ...............max_depth=3, min_samples_leaf=40; total time=   0.0s
[CV 5/5] END ...............max_depth=3, min_samples_leaf=40; total time=   0.0s
[CV 1/5] END ...............max_depth=3, min_samples_leaf=50; total time=   0.0s
[CV 2/5] END ...............max_depth=3, min_samples_leaf=50; total time=   0.0s
[CV 3/5] END ...............max_depth=3, min_samples_leaf=50; total time=   0.0s
[CV 4/5] END ...............max_depth=3, min_samples_leaf=50; total time=   0.0s
[CV 5/5] END ...............max_depth=3, min_samples_leaf=50; total time=   0.0s
[CV 1/5] END ...............max_depth=3, min_samples_leaf=60; total time=   0.0s
[CV 2/5] END ...............

[CV 1/5] END ...............max_depth=5, min_samples_leaf=50; total time=   0.0s
[CV 2/5] END ...............max_depth=5, min_samples_leaf=50; total time=   0.0s
[CV 3/5] END ...............max_depth=5, min_samples_leaf=50; total time=   0.0s
[CV 4/5] END ...............max_depth=5, min_samples_leaf=50; total time=   0.0s
[CV 5/5] END ...............max_depth=5, min_samples_leaf=50; total time=   0.0s
[CV 1/5] END ...............max_depth=5, min_samples_leaf=60; total time=   0.0s
[CV 2/5] END ...............max_depth=5, min_samples_leaf=60; total time=   0.0s
[CV 3/5] END ...............max_depth=5, min_samples_leaf=60; total time=   0.0s
[CV 4/5] END ...............max_depth=5, min_samples_leaf=60; total time=   0.0s
[CV 5/5] END ...............max_depth=5, min_samples_leaf=60; total time=   0.0s
[CV 1/5] END ...............max_depth=5, min_samples_leaf=70; total time=   0.0s
[CV 2/5] END ...............max_depth=5, min_samples_leaf=70; total time=   0.0s
[CV 3/5] END ...............

[CV 3/5] END ...............max_depth=7, min_samples_leaf=60; total time=   0.0s
[CV 4/5] END ...............max_depth=7, min_samples_leaf=60; total time=   0.0s
[CV 5/5] END ...............max_depth=7, min_samples_leaf=60; total time=   0.0s
[CV 1/5] END ...............max_depth=7, min_samples_leaf=70; total time=   0.0s
[CV 2/5] END ...............max_depth=7, min_samples_leaf=70; total time=   0.0s
[CV 3/5] END ...............max_depth=7, min_samples_leaf=70; total time=   0.0s
[CV 4/5] END ...............max_depth=7, min_samples_leaf=70; total time=   0.0s
[CV 5/5] END ...............max_depth=7, min_samples_leaf=70; total time=   0.0s
[CV 1/5] END ...............max_depth=7, min_samples_leaf=80; total time=   0.0s
[CV 2/5] END ...............max_depth=7, min_samples_leaf=80; total time=   0.0s
[CV 3/5] END ...............max_depth=7, min_samples_leaf=80; total time=   0.0s
[CV 4/5] END ...............max_depth=7, min_samples_leaf=80; total time=   0.0s
[CV 5/5] END ...............

[CV 3/5] END ...............max_depth=9, min_samples_leaf=70; total time=   0.0s
[CV 4/5] END ...............max_depth=9, min_samples_leaf=70; total time=   0.0s
[CV 5/5] END ...............max_depth=9, min_samples_leaf=70; total time=   0.0s
[CV 1/5] END ...............max_depth=9, min_samples_leaf=80; total time=   0.0s
[CV 2/5] END ...............max_depth=9, min_samples_leaf=80; total time=   0.0s
[CV 3/5] END ...............max_depth=9, min_samples_leaf=80; total time=   0.0s
[CV 4/5] END ...............max_depth=9, min_samples_leaf=80; total time=   0.0s
[CV 5/5] END ...............max_depth=9, min_samples_leaf=80; total time=   0.0s
[CV 1/5] END ...............max_depth=9, min_samples_leaf=90; total time=   0.0s
[CV 2/5] END ...............max_depth=9, min_samples_leaf=90; total time=   0.0s
[CV 3/5] END ...............max_depth=9, min_samples_leaf=90; total time=   0.0s
[CV 4/5] END ...............max_depth=9, min_samples_leaf=90; total time=   0.0s
[CV 5/5] END ...............

GridSearchCV(cv=5, estimator=DecisionTreeClassifier(),
             param_grid={'max_depth': range(1, 10),
                         'min_samples_leaf': range(10, 101, 10)},
             verbose=3)

In [64]:
model_gs.cv_results_

{'mean_fit_time': array([0.00778246, 0.00538464, 0.00585284, 0.00918469, 0.00944052,
        0.00594869, 0.00569906, 0.00682745, 0.00561275, 0.00633297,
        0.0074883 , 0.00816298, 0.00942178, 0.00733948, 0.00710788,
        0.00755939, 0.01126833, 0.0137321 , 0.01021299, 0.01159892,
        0.00907764, 0.00936837, 0.00977397, 0.01190619, 0.01206374,
        0.00966787, 0.00929089, 0.00840902, 0.01007295, 0.01034322,
        0.01401668, 0.01130767, 0.01057944, 0.0137249 , 0.01116333,
        0.01041651, 0.01063037, 0.01453433, 0.01045303, 0.01218657,
        0.01414766, 0.01534381, 0.0131022 , 0.01529183, 0.01471157,
        0.01146741, 0.0143322 , 0.01338406, 0.01356902, 0.01453218,
        0.01355762, 0.01341095, 0.0170959 , 0.01603479, 0.01575832,
        0.01594677, 0.01463284, 0.01663413, 0.01234231, 0.01598024,
        0.0169416 , 0.01900287, 0.02201428, 0.01776233, 0.0182631 ,
        0.01607785, 0.01554976, 0.01365919, 0.01293349, 0.01627812,
        0.01847181, 0.01765337,

In [71]:
model_gs.cv_results_['params']

[{'max_depth': 1, 'min_samples_leaf': 10},
 {'max_depth': 1, 'min_samples_leaf': 20},
 {'max_depth': 1, 'min_samples_leaf': 30},
 {'max_depth': 1, 'min_samples_leaf': 40},
 {'max_depth': 1, 'min_samples_leaf': 50},
 {'max_depth': 1, 'min_samples_leaf': 60},
 {'max_depth': 1, 'min_samples_leaf': 70},
 {'max_depth': 1, 'min_samples_leaf': 80},
 {'max_depth': 1, 'min_samples_leaf': 90},
 {'max_depth': 1, 'min_samples_leaf': 100},
 {'max_depth': 2, 'min_samples_leaf': 10},
 {'max_depth': 2, 'min_samples_leaf': 20},
 {'max_depth': 2, 'min_samples_leaf': 30},
 {'max_depth': 2, 'min_samples_leaf': 40},
 {'max_depth': 2, 'min_samples_leaf': 50},
 {'max_depth': 2, 'min_samples_leaf': 60},
 {'max_depth': 2, 'min_samples_leaf': 70},
 {'max_depth': 2, 'min_samples_leaf': 80},
 {'max_depth': 2, 'min_samples_leaf': 90},
 {'max_depth': 2, 'min_samples_leaf': 100},
 {'max_depth': 3, 'min_samples_leaf': 10},
 {'max_depth': 3, 'min_samples_leaf': 20},
 {'max_depth': 3, 'min_samples_leaf': 30},
 {'max_de

In [66]:
model_gs.cv_results_['mean_test_score']

array([0.614     , 0.614     , 0.614     , 0.614     , 0.614     ,
       0.614     , 0.614     , 0.614     , 0.614     , 0.614     ,
       0.66142857, 0.66142857, 0.66142857, 0.66142857, 0.66142857,
       0.66142857, 0.66142857, 0.66142857, 0.66142857, 0.66142857,
       0.692     , 0.69314286, 0.69314286, 0.69314286, 0.69314286,
       0.69314286, 0.69314286, 0.69314286, 0.69314286, 0.69514286,
       0.68942857, 0.68857143, 0.68942857, 0.68971429, 0.68942857,
       0.68914286, 0.69285714, 0.69342857, 0.69742857, 0.69314286,
       0.686     , 0.69285714, 0.69514286, 0.69314286, 0.68857143,
       0.688     , 0.69057143, 0.69085714, 0.69771429, 0.69342857,
       0.67857143, 0.68428571, 0.68714286, 0.68657143, 0.69114286,
       0.68742857, 0.69142857, 0.68971429, 0.69571429, 0.69228571,
       0.674     , 0.688     , 0.70057143, 0.69628571, 0.68971429,
       0.686     , 0.69      , 0.69114286, 0.69542857, 0.69228571,
       0.66571429, 0.688     , 0.69371429, 0.694     , 0.69228

In [67]:
model_gs.best_params_

{'max_depth': 7, 'min_samples_leaf': 30}

In [68]:
model_gs.best_score_

0.7005714285714285

In [70]:
pred = model_gs.predict(x_val)

print(confusion_matrix(y_val,pred))
print('-' * 50)
print(classification_report(y_val,pred))

[[517 221]
 [261 501]]
--------------------------------------------------
              precision    recall  f1-score   support

           0       0.66      0.70      0.68       738
           1       0.69      0.66      0.68       762

    accuracy                           0.68      1500
   macro avg       0.68      0.68      0.68      1500
weighted avg       0.68      0.68      0.68      1500

